In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [5]:

spark = SparkSession.builder \
    .appName("Lendo Parquet com PySpark") \
    .getOrCreate()

# Carregar o arquivo CSV em um DataFrame
clientes = spark.read.load("dados/Clientes.parquet")
vendas = spark.read.load("dados/Vendas.parquet")

In [6]:
from pyspark.sql import functions as Func

In [7]:
#1 Crie uma consulta que mostre, nesta ordem, Cliente, Estados e Status
clientes.select("Cliente","Estado","Status").show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [8]:
#2 Crie uma consulta que mostre apenas os clientes do Status “platinum” e “gold”
clientes.select("*").where( (Func.col("Status") == "Gold") | (Func.col("Status") == "Platinum") ).show()

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



In [12]:
# Unir os DataFrames de vendas e clientes pelo campo ClienteID
df_joined = vendas.join(clientes, vendas.ClienteID == clientes.ClienteID)

# Agrupar pelos status dos clientes e somar as vendas para cada grupo
df_result = df_joined.groupBy(clientes.Status).agg(Func.sum("Total").alias("Total_Vendas"))

# Ordenar o resultado pelo total de vendas em ordem decrescente
df_result.orderBy(Func.col("Total_Vendas").desc()).show()

+--------+------------------+
|  Status|      Total_Vendas|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+

